In [1]:
#best fitting results 

In [ ]:
install.packages("data.table")
install.packages("magrittr")

In [ ]:
library(data.table)
library(magrittr)

In [ ]:
p.threshold <- c(0.001,0.05,0.1,0.2,0.3,0.4,0.5)
#OK

In [ ]:
phenotype <- fread("final_phenotypes.txt")
#OK

In [ ]:
colnames(phenotype) <- c('FID','IID','IBDvsCON')
#OK

In [ ]:
pcs <- fread("final_IBP_6_PCs.eigenvec", header=F) %>%
    setnames(., colnames(.), c("FID", "IID", paste0("PC",1:6)) )
# The first command reads in the dataset, the command "header=F" specifies 
# that there are no column names associated with the dataset.

# The default output from plink does not include a header
# To make things simple, we will add the appropriate headers
# (1:6 because there are 6 PCs)

#OK

In [ ]:
covariate <- fread("final_IBP.cov")
# Read in the covariates (here, it is sex)
#OK

In [ ]:
pheno <- merge(phenotype, covariate) %>%
        merge(., pcs)
# Now merge the files


In [ ]:
# We can then calculate the null model (model with PRS) 
# using a linear regression 
# (as height is quantitative)
# And the R2 of the null model is 
null.r2 <- summary(lm(Height~., data=pheno[,-c("FID", "IID")]))$r.squared

In [ ]:
prs.result <- NULL
for(i in p.threshold){     # Go through each p-value threshold
    pheno.prs <- paste0("final_IBP.", i, ".profile") %>%
        fread(.) %>%
        .[,c("FID", "IID", "SCORE")] %>%
        merge(., pheno, by=c("FID", "IID"))
    # Merge the prs with the phenotype matrix
    # We only want the FID, IID and PRS from the PRS file, therefore we only select the 
    # relevant columns
    model <- lm(Height~., data=pheno.prs[,-c("FID","IID")]) %>%
            summary
    # Now perform a linear regression on Height with PRS and the covariates
    # ignoring the FID and IID from our model
  
        # model R2 is obtained as: 
    model.r2 <- model$r.squared
    prs.r2 <- model.r2-null.r2
    # We can also obtain the coeffcient and p-value of association of PRS as follow
    
    prs.coef <- model$coeff["SCORE",]
    prs.result %<>% rbind(.,
        data.frame(Threshold=i, R2=prs.r2, 
                    P=as.numeric(prs.coef[4]), 
                    BETA=as.numeric(prs.coef[1]),
                    SE=as.numeric(prs.coef[2])))
}


In [ ]:
print(prs.result[which.max(prs.result$R2),])
# Best result is:

In [ ]:
q() # exit R